# Creating MUSDBHQ Artifical Bleed Dataset

In [1]:
import librosa as lb

In [44]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plot
import os
import soundfile as sf

In [31]:
from pydub import AudioSegment
from tqdm import tqdm

### Read files, choose first 12 files alone and restricting to 2min

In [39]:
path = "/home/rajesh/Desktop/Datasets/musdb18hq/train/"
files = [path+i+'/' for i in os.listdir(path)]
files = [files[i] for i in range(13)]

In [46]:
dest_path = '/home/rajesh/Desktop/Datasets/revised_musdbbleed/Actual/'

In [47]:
sample_rate = 8000
minutes_to_crop = 2

In [49]:
vocals, bass = [], []

k = 1
for i in tqdm(files):
    vpath = i+'vocals.wav'
    bpath = i+'bass.wav'
    
    vocal, fs = lb.load(vpath, sr=sample_rate) #Downsampling for lesser memory
    bass, fs = lb.load(bpath, sr=sample_rate)
    
    scale = sample_rate*60*minutes_to_crop
    vocal = vocal[:scale] #Clipping to 2 minutes. 8000*60*2
    bass = bass[:scale]
    
    if len(vocal) == scale and len(bass) == scale:
        sf.write(dest_path+'vocal{}.wav'.format(str(k)), vocal, sample_rate)
        sf.write(dest_path+'bass{}.wav'.format(str(k)), bass, sample_rate)
        k+=1
        if k>=11:
            break

 92%|███████████████████████████████████████▋   | 12/13 [00:34<00:02,  2.88s/it]


### Generating bleed and creating 100 files out of 10 each

In [60]:
files = [dest_path+i for i in sorted(os.listdir(dest_path))]

In [68]:
save_path = '/home/rajesh/Desktop/Datasets/revised_musdbbleed/AIbleed/'

In [76]:
bass_files, vocal_files = [], []
for i in files:
    start = i.split('/')[-1][0]
    if start == 'b':
        bass_files.append(i)
    else:
        vocal_files.append(i)

In [82]:
ic = 0
for i in tqdm(vocal_files):
    s1 = AudioSegment.from_file(i, format="wav")
    for j in bass_files:
        vdname = (i.split('/')[-1]).split('.')[0]
        bbname = (j.split('/')[-1])
        bdname = (j.split('/')[-1]).split('.')[0]
        vbname = (i.split('/')[-1])
        s2 = AudioSegment.from_file(j, format="wav")
        
        if ic%2 == 0:
            dominant = s1+20
            bleed = s2-5
            vocal_flag = True
        else:
            dominant = s2+20
            bleed = s1-2
            vocal_flag = False
            
        out = dominant.overlay(bleed, position=0)
        if vocal_flag:
            out.export(save_path+vdname+bbname, format="wav")
        else:
            out.export(save_path+bdname+vbname, format="wav")
            
    ic+=1


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  9.80it/s]


### Saving it into .npy file as train & test

In [85]:
import re

In [83]:
files = os.listdir(save_path)

In [92]:
gt_path = dest_path
bleed_path = save_path

In [94]:
train, test, gt_train, gt_test = [], [], [], []

count = 1
for i in tqdm(files):
    name = re.split('(\d+)', i)
    dominant = name[0]+name[1]+name[-1]
    #bleed = name[2]+name[3]+name[-1]
    #print(i, dominant)
    
    bleeded = bleed_path+i
    gt = gt_path + dominant
    
    b, fs = lb.load(bleeded)
    g, fs = lb.load(gt)
    
    stft_b = lb.stft(b)
    stft_g = lb.stft(g)
    
    if count<=80:
        train.append(stft_b)
        gt_train.append(stft_g)
    else:
        test.append(stft_b)
        gt_test.append(stft_g)
        
    count+=1

100%|█████████████████████████████████████████| 100/100 [01:52<00:00,  1.13s/it]


In [95]:
train = np.array(train)
gt_train = np.array(gt_train)
test = np.array(test)
gt_test = np.array(gt_test)

In [96]:
train.shape, gt_train.shape, test.shape, gt_test.shape

((80, 1025, 5168), (80, 1025, 5168), (20, 1025, 5168), (20, 1025, 5168))

In [98]:
outpath = '/home/rajesh/Desktop/Datasets/revised_musdbbleed/'
np.save(outpath+'train.npy', train)
np.save(outpath+'gt_train.npy', gt_train)
np.save(outpath+'test.npy', test)
np.save(outpath+'gt_test.npy', gt_test)